In [3]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [11]:
ws_driver  = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")
ner_driver = CkipNerChunker(model="bert-base")

In [12]:
ws_driver = CkipWordSegmenter(device=-1)

In [27]:
text = [
   "病人因上述病因於107年09月18日至急診，接受心臟移植手術（1.5公分縫2針）離院。於107年09月20日及107年09月27日至門診追蹤"
]

ws  = ws_driver(text)
pos = pos_driver(ws)
ner = ner_driver(text)

Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.37it/s]


In [28]:
# Enable sentence segmentation
ws  = ws_driver(text, use_delim=True)
ner = ner_driver(text, use_delim=True)

# Disable sentence segmentation
pos = pos_driver(ws, use_delim=False)

# Use new line characters and tabs for sentence segmentation
pos = pos_driver(ws, delim_set='\n\t')

Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.37it/s]


In [29]:
# Sets the batch size and maximum sentence length
ws = ws_driver(text, batch_size=256, max_length=128)

Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.89it/s]


In [30]:
# Pack word segmentation and part-of-speech results
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        res.append(f"{word_ws}({word_pos})")
    return "\u3000".join(res)

# Show results
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
    print(sentence)
    print(pack_ws_pos_sentece(sentence_ws, sentence_pos))
    for entity in sentence_ner:
        print(entity)
    print()

病人因上述病因於107年09月18日至急診，接受心臟移植手術（1.5公分縫2針）離院。於107年09月20日及107年09月27日至門診追蹤
病人(Na)　因(P)　上述(Na)　病因(Na)　於(P)　107年(Nd)　09月(Neu)　18日(Neu)　至(P)　急診(VB)　，(COMMACATEGORY)　接受(VC)　心臟(Na)　移植(VC)　手術(Na)　（(PARENTHESISCATEGORY)　1.5(Neu)　公分(Nf)　縫(VC)　2(Neu)　針(Nf)　）(PARENTHESISCATEGORY)　離(VA)　院(Nc)　。(PERIODCATEGORY)　於(P)　107年(Neu)　09月(Neu)　20日(Neu)　及(Caa)　107年(Neu)　09月(Neu)　27日(Neu)　至(P)　門診(VA)　追蹤(VC)
NerToken(word='107年09月18日', ner='DATE', idx=(8, 18))
NerToken(word='1.5公分', ner='QUANTITY', idx=(31, 36))
NerToken(word='2', ner='CARDINAL', idx=(37, 38))
NerToken(word='107年09月20日', ner='DATE', idx=(44, 54))
NerToken(word='107年09月27日', ner='DATE', idx=(55, 65))

